In [1]:
%%time
import pal
import json
import random
from datetime import datetime
from dateutil.relativedelta import relativedelta, MO
from pal.prompt import math_prompts, colored_object_prompt, penguin_prompt, date_understanding_prompt, algorithmic_prompt
from tqdm.notebook import tqdm

CPU times: user 336 ms, sys: 4.09 ms, total: 340 ms
Wall time: 502 ms


In [61]:
today = datetime(2015, 1, 1)
today = today - relativedelta(hours=36)
#print(type(today))
print(str(today.month) + '/' + str(today.day) + '/' + str(today.year))
today = today + relativedelta(days=7)
#print(type(today))
print(str(today.month) + '/' + str(today.day) + '/' + str(today.year))
today = datetime(2001, 2, 28) + relativedelta(years=16)
# Yesterday,
yesterday = today - relativedelta(days=1)
# The answer formatted with %m/%d/%Y is
yesterday.strftime('%m/%d/%Y')

12/30/2014
1/6/2015


'02/27/2017'

In [68]:


interface = pal.interface.ProgramInterface(
  model='gpt-4o-mini',
  stop='\n\n\n', # stop generation str for Codex API
  get_answer_expr='solution()' # python expression evaluated after generated code to obtain answer 
)

prompt = "Whenever you synthesize information and perform reasoning, it is a Thought. When you need to find a date, it is an Action. There are 6 types of actions: incYear[M;D;Y;n], incMonth[M;D;Y;n], incDay[M;D;Y;n], decYear[M;D;Y;n], decMonth[M;D;Y;n], decDay[M;D;Y;n]. M, D, and Y are the Month, Day, and Year as an integer, respectively. n is the number of Years/Months/Days that you want to increment/decrement by. For example, if you wish to find the date 2 months after May 13th, 1973, you would perform the action incMonth[5;13;1973;2]. Terminate your response after an action. An Observation will be returned by the user. You will be given a question and you will perform Thoughts and Actions until you solve the question. When you get the answer, you will perform the Action called Finish and write Finish[answer]. While you are reasoning out solutions, continually refer back to relevant portions of the question and logically reason out and determine your next action. Let's think step by step."

json_list = None
max_count = 40
counter = 0
responses = []
with open('datasets/date_understanding.json', 'r') as json_file:
    json_list = json.load(json_file)
json_list = json_list["examples"]
jsonSamples = random.sample(range(0, len(json_list)), max_count)
#jsonSamples = [120, 294, 194, 131, 364, 196, 87]
#jsonSamples = [819, 1000, 699, 749, 580, 366, 1275, 203, 492, 773]
#jsonSamples = [269, 436, 836, 1291, 1283, 712, 391, 474, 1164, 783]
#jsonSamples = [263, 88, 290, 351, 196]
#print(jsonSamples)

for json_pos in tqdm(range(len(json_list)), desc="Going through date understanding questions"):
    result = json_list[json_pos]
    question = result["input"]
    target = result["target_scores"]
    curResponse = {"question": question, "target": target}
    messages=[
        {"role": "system", "content": "You are a helpful assistant that accurately solves reasoning questions about dates by acting as an agent who will perform Thoughts and Actions."},
#        {"role": "user", "content": prompt + '\n' + "Question: Fred remembers that on his 2nd day of being married, September 7th, he saw a calendar for the year. The year on the calendar was 2015. Today is his 2 year anniversary. What is the date today?"},
#        {"role": "assistant", "content": """Thought: Fred's 2nd day of being married was on September 7th. The year on the calendar was 2015, so his 2nd day of being married must have been September 7th, 2015. Thus, the day that he got married was 1 day before 9/7/2015. Thus, I first need to find the day that is before 9/7/2015. 
#Action: decDay[9;7;2015;1]"""},
#        {"role": "user", "content": "Observation: 9/6/2015"},
#        {"role": "assistant", "content": """Thought: So the day he got married was 9/6/2015. Today is his 2 year anniversary, so today's date is two years after 9/6/2015. 
#Action: incYear[9;6;2015;2]"""},
#        {"role": "user", "content": "Observation: 9/6/2017"},
#        {"role": "assistant", "content": """Thought: So today is 9/6/2017. The question asks what date is today, so that is my answer.
#Action: FINISH[09/06/2017]"""},
#        {"role": "user", "content": prompt + '\n' + "Question: The date in 6 months will be October 18th, 2016. What will be the date in 53 days?"},
#        {"role": "assistant", "content": """Thought: If the date in 6 months is October 18th, 2016, then today is 6 months before October 18th, 2016. Thus, I need to find the date 6 months before 10/18/2016.
#Action: decMonth[10;18;2016;6]"""},
#        {"role": "user", "content": "Observation: 4/18/2016"},
#        {"role": "assistant", "content": """Thought: So today is 4/18/2016. The question asks what the date will be in 53 days, so I need to find the date that is 53 days after 4/18/2016. 
#Action: incDay[4;18;2016;53]"""},
#        {"role": "user", "content": "Observation: 6/10/2016"},
#        {"role": "assistant", "content": """Thought: So the date in 53 days is 6/10/2016.
#Action: FINISH[06/10/2016]"""},
        {"role": "user", "content": prompt + '\n' + "Question: 2015 is coming in 36 hours. What is the date one week from today in MM/DD/YYYY?"},
        {"role": "assistant", "content": """Thought: If 2015 is coming in 36 hours, then today must be two days before 1/1/2015. I must first find today's date then.
Action: decDay[1;1;2015;2]"""},
        {"role": "user", "content": "Observation: 12/30/2014"},
        {"role": "assistant", "content": """Thought: Therefore, today is 12/30/2014. The question asks what the date one week from today is, so I must find the date that is 7 days after 12/30/2014.
Action: incDay[12;30;2014]"""},
        {"role": "user", "content": "Observation: 1/6/2015"},
        {"role": "assistant", "content": """Thought: Therefore, the date one week from today is 1/6/2015.
Action: FINISH[01/06/2015]"""},
        {"role": "user", "content": prompt + '\n' + "Question: The first day of 2019 is a Tuesday, and today is the first Monday of 2019. What is the date today in MM/DD/YYYY?"},
        {"role": "assistant", "content": """Thought: If the first day of 2019 is a Tuesday, and today is the first Monday of 2019, then today is 6 days later. Thus, to find the date today, I need to find the date 6 days after 1/1/2019.
Action: incDay[1;1;2019;6]"""},
        {"role": "user", "content": "Observation: 1/7/2019"},
        {"role": "assistant", "content": """Thought: Therefore, the date today is 1/7/2019. The question asks for the date today, so this is my answer.
Action: FINISH[01/07/2019]"""},
        {"role": "user", "content": prompt + '\n' + "Question: Jane was born on the last day of Feburary in 2001. Today is her 16-year-old birthday. What is the date yesterday in MM/DD/YYYY?"},
        {"role": "assistant", "content": """Thought: If Jane was born on the last day of Feburary in 2001 and today is her 16-year-old birthday, then today is 16 years later. The last day of February in 2001 was 2/28/2001, so I first need to find the date 16 years after 2/28/2001.
Action: incYear[2;28;2001;16]"""},
        {"role": "user", "content": "Observation: 2/28/2017"},
        {"role": "assistant", "content": """Thought: Therefore, today is 2/28/2017. The question asks for the date yesterday, so I need to find the day before 2/28/2017.
Action: decDay[2;28;2017;1]"""},
        {"role": "user", "content": "Observation: 2/27/2017"},
        {"role": "assistant", "content": """Thought: Therefore, the date yesterday was 2/27/2017, which is what the question asks for.
Action: FINISH[02/27/2017]"""},
        {"role": "user", "content": prompt + '\n' + "Question: " + question}
    ]
    #for message in messages:
    #    print(message["role"].upper())
    #    print(message["content"])
    #    print("")
    #break
    # interface.verbose = True
    finished = False
    #print(question)
    #print("Target: " + str(target))
    numIter = 0
    maxIter = 20
    while not finished:
        if numIter >= maxIter:
            curResponse["received"] = -98765421
            print("Maximum number of iterations reached")
            print("#####################################")
            break
        gens = interface.generate_history(messages=messages, max_tokens=1024)
        output = gens[0]
        #print(output)
        messages.append({"role": "assistant", "content": output})
        curAction = output[output.find("Action: ") + len("Action: "):]
        curAction = curAction.lower()
        expression = curAction[curAction.find('[') + 1:curAction.find(']')]
        expression = expression.replace(',', '') # Commas are interpreted as tuples, so we remove them to avoid errors with eval
        #print(curAction)
        if "finish" in curAction:
            finished = True
            curResponse["received"] = expression
        else:
            expression = expression.split(';')
            try:
                curDay = datetime(int(expression[2]), int(expression[0]), int(expression[1]))
                num = int(expression[3])
            except:
                messages.append({"role": "user", "content": "Observation: That was not a valid action. Please input a valid one."})
            if "incday" in curAction:
                curDay = curDay + relativedelta(days=num)
            elif "incmonth" in curAction:
                curDay = curDay + relativedelta(months=num)
            elif "incyear" in curAction:
                curDay = curDay + relativedelta(years=num)
            elif "decday" in curAction:
                curDay = curDay - relativedelta(days=num)
            elif "decmonth" in curAction:
                curDay = curDay - relativedelta(months=num)
            elif "decyear" in curAction:
                curDay = curDay - relativedelta(years=num)
            else:
                evaled = "That was not a valid action. Please input a valid one."
            messages.append({"role": "user", "content": "Observation: " + str(curDay.month) + '/' + str(curDay.day) + '/' + str(curDay.year)})
            #print(messages[-1]["content"])
    curResponse["messages"] = messages
    responses.append(curResponse)
    counter += 1

numCorrect = 0
curCount = 0
for response in responses:
    #print("Problem " + str(curCount))
    #print("Received: " + response["received"])
    #print("Target: ", end="")
    #print(response["target"])
    #print("")
    curCount += 1
    #for message in response["messages"]:
    #    print(message["content"])
    if response["received"] not in response["target"].keys():
        continue
    if response["target"][response["received"]] == 1:
        numCorrect += 1
print("Number correct = " + str(numCorrect))
with open('date-prompting-results-01-gpt-4o-mini.json', 'w') as f:
    json.dump(responses, f)

Going through date understanding questions:   0%|          | 0/369 [00:00<?, ?it/s]

Number correct = 323


In [44]:
for message in responses[1]["messages"]:
    print(message["content"])

You are a helpful assistant that accurately solves reasoning questions about dates by acting as an agent who will perform Thoughts and Actions.
Whenever you synthesize information and perform reasoning, it is a Thought. When you need to find a date, it is an Action. There are 6 types of actions: incYear[M;D;Y;n], incMonth[M;D;Y;n], incDay[M;D;Y;n], decYear[M;D;Y;n], decMonth[M;D;Y;n], decDay[M;D;Y;n]. M, D, and Y are the Month, Day, and Year as an integer, respectively. n is the number of Years/Months/Days that you want to increment/decrement by. For example, if you wish to find the date 2 months after May 13th, 1973, you would perform the action incMonth[5, 13, 1973, 2]. Terminate your response after an action. Then, an Observation is returned by the user. You will be given a question and you will perform Thoughts and Actions until you get the answer. When you get the answer, you will perform the Action called Finish and write Finish[answer]. While you are reasoning out solutions, ens

In [5]:
print(len(json_list))

369
